In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import metrics
import math
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

df = pd.read_csv('/kaggle/input/wine-quality/winequalityN.csv')
col_list = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','pH','sulphates']
type_cls = df['type'].unique()
df.loc[(df['quality']<5),'quality'] = 1
df.loc[(df['quality']>6),'quality'] = 3
df.loc[((df['quality']==5) | (df['quality']==6)),'quality'] = 2
df['quality'].value_counts()


# Replacing missing values with the class mean

In [ ]:
df1 = df
grp = df.groupby('type')
for coln in col_list:
    for cls in type_cls:
      filt = (df[coln].isnull()) & (df['type'] == cls)
      df.loc[filt,coln] = grp[coln].mean().loc[cls]   
df = pd.get_dummies(df)
test_size = int(math.ceil(len(df)*.1))
df_test = df[-test_size:]
df = df[:-test_size]
y = np.array(df['quality'])
X = (df.drop('quality',1))
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size = .1)

In [ ]:
pipe = Pipeline([('std',StandardScaler()),
               ('clf',KNeighborsClassifier(n_neighbors=5))])
#clf = KNeighborsClassifier(n_neighbors=11)
pipe.fit(X_train,Y_train)
pipe.score(X_test,Y_test)
pred_y = df_test['quality']
pred = pipe.predict(df_test.drop('quality',1))
metrics.accuracy_score(pred_y,pred)

In [ ]:
#df1 = df
#for j,colm in enumerate(col_list):
#    for name,group in grp:
#       df.loc[(df[colm].isnull()), colm] = grp[colm].mean().loc[name]
#       print(type(df.loc[(df[colm].isnull()), colm]))
#df.isnull().sum()    

# Below visualisation is just to compare the distribution of original vs cleaned data to make sure cleaning doesn't have much impact on distribution.

In [ ]:
#Visualization
fig = plt.figure(figsize=(70,70))
#sns.set_palette('hus1')
for i,col in enumerate(col_list):
    plt.subplot2grid((len(col_list),2),(i,0),rowspan = 1, colspan = 2)
    sns.distplot(df1[col],color="red", label="original")
    plt.legend()
    plt.subplot2grid((len(col_list),2),(i,0),rowspan = 1, colspan = 2)
    sns.distplot(df[col],color="black", label="corrected")
    plt.legend()
plt.savefig('/kaggle/working/plot.png')
plt.show()    